# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [ ]:
import pandas as pd

# Notebook Presentation

In [ ]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [ ]:
df_apps = pd.read_csv('apps.csv', low_memory=False)

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [ ]:
df_apps.shape

In [ ]:
df_apps.columns

In [ ]:
df_apps.sample(5)

### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns.

In [ ]:
df_apps = df_apps.drop(['Last_Updated', 'Android_Ver'], axis=1)
df_apps

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows.

In [ ]:
df_apps['Rating'].isna().sum()

In [ ]:
df_apps.isna().any(axis=1).sum() # number of rows with at least one missing value

In [ ]:
df_apps_clean = df_apps.dropna() # removes every row with a missing value

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`.


In [ ]:
df_apps_clean.duplicated().sum()

In [ ]:
df_apps_clean[df_apps_clean['App'] == 'Instagram']

In [ ]:
df_apps_clean.shape

In [ ]:
df_apps_clean = df_apps_clean.drop_duplicates(['App','Type','Price'])

In [ ]:
df_apps_clean.shape

In [ ]:
df_apps_clean[df_apps_clean['App'] == 'Instagram']

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [ ]:
df_apps_clean.sort_values(by='Rating', ascending=False).head(20)

# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please?

In [ ]:
df_apps_clean.sort_values(by='Size_MBs', ascending=False).head()

# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [ ]:
df_apps_clean.sort_values(by='Reviews', ascending=False).head(50)

# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [ ]:
ratings = df_apps_clean['Content_Rating'].value_counts()
ratings

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
# convert index to numpy array to avoid warning message
fig = px.pie(labels=ratings.index.to_numpy(), values=ratings.values, title='Content Rating', names=ratings.index.to_numpy())
fig.update_traces(textposition='outside', textinfo='label+percent')
fig.show();

In [ ]:
fig = px.pie(labels=ratings.index.to_numpy(), values=ratings.values, title='Content Rating', names=ratings.index.to_numpy(), hole=0.6)
fig.update_traces(textposition='outside', textinfo='label+percent')
fig.show()

# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install?

Check the datatype of the Installs column.

Count the number of apps at each level of installations.

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first.

In [ ]:
df_apps_clean['Installs'].unique()

In [ ]:
df_apps_clean['Installs'].value_counts()

In [ ]:
df_apps_clean['Installs'] = df_apps_clean['Installs'].str.replace(',','').apply(int)

In [ ]:
df_apps_clean['Installs'].value_counts().sort_index()

# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [ ]:
df_apps_clean['Price'] = df_apps_clean['Price'].str.replace('$','',regex=False).apply(float)

### The most expensive apps sub $250

In [ ]:
df_apps_clean = df_apps_clean[df_apps_clean['Price'] <= 250]

### Highest Grossing Paid Apps (ballpark estimate)

In [ ]:
df_apps_clean['Revenue_Estimate'] = df_apps_clean['Price'] * df_apps_clean['Installs']

In [ ]:
df_apps_clean.sort_values(by='Revenue_Estimate', ascending=False).head(10)

# Plotly Bar Charts & Scatter Plots: Analysing App Categories

In [ ]:
df_apps_clean['Category'].nunique()

In [ ]:
top10_category = df_apps_clean['Category'].value_counts()
top10_category

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [ ]:
bar = px.bar(x=top10_category.index, y=top10_category.values)
bar.show()

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [ ]:
category_installs = df_apps_clean.groupby('Category').agg({'Installs': pd.Series.sum})
category_installs = category_installs.sort_values('Installs', ascending=True)

In [ ]:
h_bar = px.bar(x=category_installs['Installs'], y=category_installs.index, orientation='h', title='Category Popularity')
h_bar.update_layout(xaxis_title='Number of Downloads', yaxis_title='Category')
h_bar.show()

### Category Concentration - Downloads vs. Competition

**Challenge**:
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this.

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log')

In [ ]:
apps_installs = pd.concat([top10_category, category_installs], axis=1).reset_index()

In [ ]:
apps_installs.columns = ['Category', 'App', 'Installs']
apps_installs.set_index('Category')

In [ ]:
scat = px.scatter(data_frame=apps_installs, x='App', y='Installs', size='App', color='Installs', hover_name='Category', title='Category Concentration')
scat.update_layout(yaxis=dict(type='log'), xaxis_title='Number of Apps (Lower=More Concentrated)')

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html).


In [ ]:
split_stack = df_apps_clean['Genres'].str.split(';', expand=True).stack()

In [ ]:
split_stack.value_counts()

In [ ]:
split_stack.value_counts().size

# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data?

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/).

* Find a way to set the colour scale using the color_continuous_scale parameter.
* Find a way to make the color axis disappear by using coloraxis_showscale.

In [ ]:
bar = px.bar(x=split_stack.value_counts().index, y=split_stack.value_counts().values, color=split_stack.value_counts().values, title='Top Genres', color_continuous_scale='Agsunset')
bar.update_layout(xaxis_title='Genre', yaxis_title='Number of Apps', coloraxis_showscale=False)
bar.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [ ]:
df_apps_clean['Type'].value_counts()

In [ ]:
df_free_vs_paid = df_apps_clean.groupby(['Category', 'Type']).count()\
                               .reset_index()[['Category','Type','App']]
df_free_vs_paid

**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart:

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category.

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value).

In [ ]:
bar_paid_vs_free = px.bar(data_frame=df_free_vs_paid, x='Category', y='App', color='Type', barmode='group', title='Free vs Paid Apps by Category')
bar_paid_vs_free.update_xaxes(categoryorder='total descending')
bar_paid_vs_free.update_layout(yaxis=dict(type='log'), yaxis_title='Number of Apps')

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart.

<img src=https://imgur.com/uVsECT3.png>


In [ ]:
box_free_vs_paid = px.box(df_apps_clean, x='Type', y='Installs', log_y=True, color='Type', points='all', title='How Many Downloads are Paid Apps Giving Up?')
box_free_vs_paid.show()

# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below:

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories.

In [ ]:
box_revenue = px.box(df_apps_clean[df_apps_clean['Type']=='Paid'], x='Category', y='Revenue_Estimate', log_y=True, title='How Much Can Paid Apps Earn?')
box_revenue.update_xaxes(categoryorder='min ascending')
box_revenue.update_layout(yaxis_title='Paid App Ballpark Revenue')
box_revenue.show()

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.